In [136]:
!pip install torch==2.5.1 -U
!pip install numpy==1.23.5 -U
!pip install huggingface-hub==0.28.1 -U
!pip install transformers==4.48.2 -U
!pip install datasets==3.2.0 -U
!pip install peft==0.14.0 -U
!pip install pandas==2.2.2 -U
!pip install trl==0.14.0 -U 
!pip install bitsandbytes==0.45.1 -U

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## This notebook can be completed on Colab. You will need a GPU to train the model. Export this notebook to colab and run the cell above this to install the necessary packages. Be sure to use a T4 GPU runtime. Also upload your train.csv, training_utils.py, and submission.py files to Colab, in the same directory as this notebook. To verify you have saved the files in the correct place, you should be able to run the import cells below with no import errors, after the packages above have been installed.

# Part 0: Following up from the previous assignment...

In the previous assignment, you had to prompt engineer Gemini 1.5 to perform arithmetic. 

It was not an entirely trivial task, despite Gemini 1.5 being a large language model, probably trained on text that explains how to do arithmetic.

In particular, Gemini struggled with large number arithmetic because there are not many of such examples on the internet, when compared to small number arithmetic.

Small language models are even worse at arithmetic, but you can fine tune them to use tools like a user-defined calculator.

In this assignment, you will train a small language model to use a simple calculator that you will build. 

## To Do:

1. Implement the `prepare_data()`, `use_calculator()`, `api_key()`, and `hub_model_name()` functions in `submission.py`
2. Read the code below and read `training_utils.py`. Take time to understand how the code works. The provided code contains the functions that are needed to perform instruction fine tuning, which does not follow the exact same process as the auto-regressive, next token prediction fine tuning method. While you will not be graded on whether you read the code, you should try to understand it for your own education.
3. Fill out the lines in `training_utils.py` that are commented with `TODO`. These are hyperparameters you need to tune. You are free to tune any of the hyperparameters, but you should be able to get a good model with just the TODO hyperparameters.
4. Run the code below to train your model. 
5. Push your model to the Hugging Face Hub with the `hub_model_name()` function. This should be in the final cell of this notebook.

Thus, this notebook is where you can test the model and push it to the hub but you will not be graded on this notebook. Your `submission.py` file will contain the code used to create the data set and your huggingface information, which we will use to pull your model for grading. `training_utils.py` contains the code used to train the model with hyperparameters you can tune to facillitate your training, but you will **not** be graded on that file either.

In [120]:
import training_utils as training_utils
import submission as submission

#### The following cell can be run after you implement part TODO 1,2 and 3 

In [ ]:
sft_config = training_utils.SFTConfig
sft_config.hf_key = submission.api_key()
model, tokenizer = training_utils.load_model(sft_config)
df = submission.prepare_data('train.csv')

#### The following cell runs TODO part 4


In [ ]:
trained_model = training_utils.train(dataset=df, config=sft_config)

# Part 1: Testing your model

The code below tests your model. It should do well on the training set since this is what you trained on.

###  Grading Criteria:

Accuracy is define as the proportion of correct answers.

Your grade will be assigned based on the accuracy of your model.

Because you are training the model to use a calculator only, the accuracy should be very close to 1.0, if not at 1.0, thus the grading criteria will have a higher bar for accuracy.

On the evaluation set, your assignment score will be calculated as follows:

- Accuracy >= 1.0: 1 * Full points
- Accuracy >= 0.98: 0.95 * Full points
- Accuracy >= 0.95: 0.9 * Full points
- Accuracy >= 0.9: 0.8 * Full points
- Accuracy >= 0.85: 0.75 * Full points
- Accuracy >= 0.5: 0.7 * Full points
- Else: 0.5 * Full points


In [ ]:
import pandas as pd
test_df = pd.read_csv('train.csv')
operations = {'add': '+', 'subtract': '-', 'multiply': '*'}
correct = 0
running_total = 0
for k, r in test_df.iterrows():
    running_total += 1
    response = training_utils.run_inference_with_model(trained_model, tokenizer, f'### Question: What is {r["num1"]} {operations[r["operation"]]} {r["num2"]}?\n ### Answer:', config=sft_config)
    try:
        if submission.capture_output(response) == r['result']:
            correct += 1
        print(f'{r["num1"]} {operations[r["operation"]]} {r["num2"]} = ', submission.capture_output(response), 'compare with ', r['result'], 'score: ', f"{correct}/{running_total}")
    except:
        print(response, 'UNABLE TO GET TOOL USE')
print(f'Accuracy: {correct/len(test_df)}')

response = training_utils.run_inference(sft_config, '### Question: What is 651475 * 4458?\n ### Answer:')

If you are struggingly to get the full accuracy, you might want to train your model on more data, or on more diverse data. 

Many methods in machine learning exist to improve the accuracy of your model, in particular such as data augmentation to encourage regularization and prevent overfitting.

Your model might learn a spurious correlation, e.g. it may be most subtraction problems begin with a number starting with a 9, so any time it sees a number starting with a 9, it will assign it a subtraction operation. To prevent this type of overfitting, you can add data to your training set that does not have this pattern. This is just an example and may not hold true for your model, but you should observe what patterns your model is learning, why it is learning them, and how can you encourage it to learn other patterns.

This means you might want to append more examples to `train.csv` to encourage your model to learn other patterns. If you choose to do this, take a look at the `train.csv` file and add new entries to the file, following the same format as the rest of the entries in the data set. You should be able to write some code to do this systematically. Keep in mind the larger the data set, the more memory and time it will take to train the model. You may also delete some of the training data if you think it is not useful for training the model.


# Part 2: Push your model to the Hugging Face Hub

When you are satisfied with your model, you will push your model to the Hugging Face Hub by giving it a name and using your API key.

The name should be in the format of `<username>/<model name>` without the brackets.

For example if your username is `johndoe` and you want the model name to be `sft-calculator`, the name should be `johndoe/sft-calculator`.  

This completes TODO part 5.

In [ ]:
trained_model.push_to_hub(submission.hub_model_name(), token=submission.api_key())